In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import funzioni as fx
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp

def grafico(data):
    target = go.Scatter(
        x = data.index,
        y = data['EMA_20'],
        mode = 'lines',
        line = dict(color='rgba(0, 200, 0, 1)'),
        name = 'EMA_20'
    )
    
    prev1 = go.Scatter(
        x = data.index,
        y = data['Previsione'],
        mode = 'lines',
        line = dict(color='rgba(250, 120, 0, .8)'),
        name = 'Previsione'
    )
    
    
    layout = dict(xaxis = dict(domain=[0, 1]),
                  yaxis = dict(title = 'Close', autorange=True, domain=[0, 1]),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
        
    fig = sp.make_subplots(rows=1, cols=1)
    fig.update_layout(layout)
    
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(prev1, row=1, col=1)
    pyo.plot(fig, filename="risultato.html", auto_open=True)



def definizione_parametri_e_target(df):
    if len(df) > 0:
        X = df[
            ["Volume"] + 
            ["EMA_20"] + 
            ["EMA_50"] + 
            ["PSARaf"] + 
            ["PSARr"] + 
            ["PSAR"] + 
            ["MACD"] + 
            ["MACDh"] + 
            ["TSI"] + 
            ["TSIs"] + 
            ["SUPERT"] + 
            ["SUPERTd"] + 
            ["ADX"] + 
            ["DM_OSC"] + 
            ["TRIX"] +
            ["TRIXs"] + 
            ["KVO"] + 
            ["KVOs"] + 
            ["VTX_OSC"] + 
            ["AROONOSC"] + 
            ["VI_OSC"] + 
            ["VHF"] + 
            ["ATR"] 
        ].copy()
        return X
    else:
        return df


def elimina_valori_non_validi(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True, axis=0)
    return df


# 1. Preparazione dei Dati
data = pd.read_parquet('../Dati_borsa/Dati_calcolati/BTG.parquet')
data = elimina_valori_non_validi(data)
data = definizione_parametri_e_target(data)

# 2. Creazione di Sequenze Temporali
look_back = 30
giorni_previsione = 20
X = []
for i in range(look_back, len(data) - giorni_previsione):
    X.append(data.iloc[i - look_back:i].values)
X = np.array(X)

# 3. Caricamento del Modello e Previsioni
model = keras.models.load_model('LSTM.keras')
predictions = model.predict(X)

# 4. Aggiungere le Previsioni come Nuove Colonne al DataFrame
data_finale = data.iloc[look_back:len(data) - giorni_previsione].copy()
data_finale["Previsione"] = predictions

# 6. Pulizia dei dati (opzionale)
data_finale.dropna(axis=0, inplace=True)


113/113 [==============================] - 9s 59ms/step


In [2]:
#data_finale.to_excel("risultato.xlsx")

In [3]:
    
grafico(data_finale)